# Settings

In [1]:
# Load Reticulate function
Sys.setenv(RETICULATE_PYTHON="/home/luca/anaconda3/envs/reticulate/bin/python")
library(reticulate)
reticulate::use_python("/home/luca/anaconda3/envs/reticulate/bin/python")
reticulate::use_condaenv("/home/luca/anaconda3/envs/reticulate")
reticulate::py_module_available(module='anndata') #needs to be TRUE
reticulate::import('anndata') #good to make sure this doesn't error
reticulate::py_module_available(module='leidenalg') #needs to be TRUE
reticulate::import('leidenalg') #good to make sure this doesn't error

[1] TRUE

Module(anndata)

[1] TRUE

Module(leidenalg)

In [2]:
## Patch for annotations in R4.1
# BiocManager::install("Bioconductor/GenomeInfoDb",lib = "/home/luca/R/x86_64-pc-linux-gnu-library/4.1",force = TRUE)
# library(GenomeInfoDb,lib.loc="/home/luca/R/x86_64-pc-linux-gnu-library/4.1")

In [16]:
# Load packages
pacman::p_load(dplyr, stringr, data.table, tidyr, data.table, Matrix, future, 
               hdf5r, Seurat, Signac,harmony, knitr, SoupX, 
               EnsDb.Hsapiens.v86, 
               logr, parallel, 
               ggplot2, ggpubr, ggrepel, ggbreak, gridExtra, patchwork, grid, ggh4x)

In [4]:
# Load genome
#suppressMessages(annotations <- GetGRangesFromEnsDb(ensdb=EnsDb.Hsapiens.v86))
#genome(annotations) <- 'hg38'
#seqlevelsStyle(annotations) <- 'UCSC'
# Save table
# writeRDS(annotations, "/nfs/lab/Luca/Assets/references/Cellranger/hg38.annotations.rds")

# Load table
annotations = readRDS("/nfs/lab/Luca/Assets/references/Cellranger/hg38.annotations.rds")
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- 'hg38'

# Seq info downloaded from: https://github.com/broadinstitute/ichorCNA/issues/84
seq.info = readRDS("/nfs/lab/Luca/Assets/references/Cellranger/seqinfo_hg38_ucsc.rds")

In [5]:
# Set options
options(stringsAsFactors = FALSE)
warnLevel <- getOption('warn')
options(warn = -1)
opts_chunk$set(tidy=TRUE)

# set Future
plan("multicore", workers = 4)
# set RAM treshold
## 1000 = 1gb
RAM.tresh = 10000 * 1024^2
options(future.globals.maxSize = RAM.tresh)

In [12]:
# Set directories
base.dir = "/nfs/lab/projects/mega_heart/"
assets.dir = "/nfs/lab/projects/mega_heart/Assets/"

lv.windows.dir = "/nfs/lab/scorban/fnih_heartLV/integration/Feb03_2024_seuratintegration/Feb03_2024_LVHeart_30donors_4lanes_mergedseuratobject_HVWs50K.txt"
rv.windows.dir = "/nfs/lab/scorban/fnih_heartRV/integration/Feb01_2024_seuratintegration/Feb01_2024_RVHeart_30donors_4lanes_mergedseuratobject_HVWs50K.txt"
la.windows.dir = "/nfs/lab/projects/mega_heart/FNIH/multiome/Analysis/0_multimodal_processing_SMC/LA/integration/May07_2024_LAHeart_30donors_4lanes_mergedseuratobject_HVWs50K.txt"
ra.windows.dir = "/nfs/lab/projects/mega_heart/FNIH/multiome/Analysis/0_multimodal_processing_SMC/RA/integration/May08_2024_RAHeart_30donors_4lanes_mergedseuratobject_HVWs50K.txt"

cellranger.links.dir = "/nfs/lab/projects/mega_heart/FNIH/multiome/cellranger.symlinks/"

step1.dir = "/nfs/lab/projects/mega_heart/FNIH/multiome/Analysis/1_preprocessing/"
step2.dir = "/nfs/lab/projects/mega_heart/FNIH/multiome/Analysis/3_downstream/Major_celltypes/"

counts.dir = paste0(step2.dir, "RNA/4Chambers/COUNTS/")
TPM.dir = paste0(step2.dir, "RNA/4Chambers/TPM/")
DESEQ.dir = paste0(step2.dir, "RNA/4Chambers/DESEQ/")
GSEA.res.dir = paste0(step2.dir, "RNA/4Chambers/DESEQ/GSEA/")

In [7]:
# Create those directories
dir.create(paste0(step2.dir, "RNA/"))
dir.create(paste0(step2.dir, "RNA/4Chambers/"))
dir.create(step2.dir)
dir.create(counts.dir)
dir.create(TPM.dir)
dir.create(DESEQ.dir)
dir.create(GSEA.res.dir)

In [8]:
log_open(file_name = paste0(base.dir, "LA_RNA_DownstreamFiles.log"))

[1] "/nfs/lab/projects/mega_heart/log/LA_RNA_DownstreamFiles.log"

# Load assay

In [9]:
log_print(" Loading data")
adata = readRDS(paste(step1.dir, "LV_RV_LA_RA.multiome.mrg.filt.MTless.silQC.curated.peaks.rds", sep = ""))
log_print(paste("Done"))

[1] " Loading data"
[1] "Done"


In [ ]:
# Backup then indent
# adata.bckp = adata

In [10]:
adata

An object of class Seurat 
392885 features across 329255 samples within 4 assays 
Active assay: ATAC (285873 features, 285873 variable features)
 2 layers present: counts, data
 3 other assays present: RNA, RNA_raw, SCT
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

# Cell count matrix  - RNA

In [11]:
samples = as.character(unique(adata$donor))
samples

[1] "DTX089" "DTX119" "D47"    "DTX027" "D36"    "DTX097" "DTX066" "DTX088"
 [9] "D55"    "DTX093" "DTX105" "DTX077" "D40"    "DTX062" "DTX060" "D38"   
[17] "DTX037" "D53"    "DTX017" "DTX114" "D52"    "DTX065" "D37"    "DTX081"
[25] "D35"    "DTX091" "DTX087" "DTX096" "DTX031" "D51"

In [12]:
######## SET TO WHATEVER YOUR ASSIGNMENTS ARE STORED UNDER ########
Idents(object = adata) <- "cell.major_types"
head(Idents(adata))
#### OUTPUT DIRECTORY #####
outdir = counts.dir

#pull out list of all cell types, removing ignore
unique_cell_types <- unique(adata$cell.major_types)
#unique_cell_types <- unique_cell_types[-c(11)]
print(unique_cell_types)


sample_bcs <- list()
for (sample in samples){
    sample_bcs[[sample]] <- row.names(adata[[]][adata[[]]$donor == sample,])
}

##############
#### SET TO WHATEVER ASSAY YOU WANT TO USE ######
DefaultAssay(adata) <- 'RNA'
gex.counts <- GetAssayData(adata, slot='counts')
dim(gex.counts)
head(gex.counts)
adata_matrices <- adata

QY_2193_1_2_QY_2192_1_2_AAACAGCCAACTAGGG-1 
                                Fibroblast 
QY_2193_1_2_QY_2192_1_2_AAACAGCCACTTACAG-1 
                               Endothelial 
QY_2193_1_2_QY_2192_1_2_AAACAGCCAGTTTGTG-1 
                                       vCM 
QY_2193_1_2_QY_2192_1_2_AAACAGCCATCCGTAA-1 
                               Endothelial 
QY_2193_1_2_QY_2192_1_2_AAACAGCCATGCATAT-1 
                                       vCM 
QY_2193_1_2_QY_2192_1_2_AAACATGCAGGCCTTG-1 
                                Fibroblast 
12 Levels: Fibroblast Endothelial vCM Myeloid Pericyte Endocardial ... aCM

 [1] "Fibroblast"  "Endothelial" "vCM"         "Myeloid"     "Pericyte"   
 [6] "Endocardial" "Lymphoid"    "SM"          "Neuronal"    "Adipocyte"  
[11] "Epicardial"  "aCM"        


[1]  36510 329255

  [[ suppressing 34 column names 'QY_2193_1_2_QY_2192_1_2_AAACAGCCAACTAGGG-1', 'QY_2193_1_2_QY_2192_1_2_AAACAGCCACTTACAG-1', 'QY_2193_1_2_QY_2192_1_2_AAACAGCCAGTTTGTG-1' ... ]]



6 x 329255 sparse Matrix of class "dgCMatrix"
                                                                               
MIR1302-2HG . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
FAM138A     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
OR4F5       . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AL627309.1  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AL627309.3  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AL627309.2  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                  
MIR1302-2HG ......
FAM138A     ......
OR4F5       ......
AL627309.1  ......
AL627309.3  ......
AL627309.2  ......

 .....suppressing 329221 columns in show(); maybe adjust options(max.print=, width=)
 ..............................

In [13]:
#looping through cell types by making ^ into a function
get_per_sample_gex_SUMS <- function(cell.type, filename){
    print(paste(cell.type,Sys.time()))

    #pull out rows of gex.counts where BC Ident matches cell.type
    bcs <- names(Idents(adata_matrices)[Idents(adata_matrices) == cell.type])
    counts <- gex.counts[,colnames(gex.counts) %in% bcs]
    print(dim(counts))

    #initialize the matrix of sample gex
    counts.df <- as.data.frame(rep(0,length(row.names(gex.counts))))
    row.names(counts.df) <- row.names(gex.counts)
    colnames(counts.df) <- c('temp')

    #go through samples and calculate sum of gex values
    for (sample in samples){
        sample_cols <- colnames(counts) %in% sample_bcs[[sample]]
        counts.cut <- counts[,sample_cols]
        
        #if only one bc, this becomes a vector which is an issue
        if (typeof(counts.cut) == 'double'){
            mean.counts <- counts.cut
        #if there are NO bcs, this will return NA (just return 0 for everything)
        } else if(length(colnames(counts.cut)) == 0){
            mean.counts <- rep(0,length(row.names(counts)))
        } else {
            mean.counts <- rowSums(counts.cut)
        }
        counts.df <- cbind(counts.df,as.data.frame(mean.counts))
     }
    fin.counts.df <- counts.df[,-c(1)]
    colnames(fin.counts.df) <- samples
    head(fin.counts.df)

    #export df
    write.table(fin.counts.df, filename, sep='\t',quote=FALSE)
}

In [14]:
##### NAME YOUR FILES #####
for (cell.type in unique_cell_types){
    filename <- paste(outdir, cell.type, '_perdonor.gex_SoupX.RNA.counts', sep = "")
    get_per_sample_gex_SUMS(cell.type, filename)
}

[1] "Fibroblast 2025-01-28 13:07:31.436829"
[1] 36510 67157
[1] "Endothelial 2025-01-28 13:07:40.942306"
[1] 36510 53650
[1] "vCM 2025-01-28 13:07:47.630199"
[1] 36510 68449
[1] "Myeloid 2025-01-28 13:08:02.537411"
[1] 36510 51355
[1] "Pericyte 2025-01-28 13:08:10.954565"
[1] 36510 22489
[1] "Endocardial 2025-01-28 13:08:14.5894"
[1] 36510  8927
[1] "Lymphoid 2025-01-28 13:08:17.917017"
[1] 36510 20021
[1] "SM 2025-01-28 13:08:21.643059"
[1] 36510  6561
[1] "Neuronal 2025-01-28 13:08:24.198413"
[1] 36510  3962
[1] "Adipocyte 2025-01-28 13:08:26.634516"
[1] 36510  1122
[1] "Epicardial 2025-01-28 13:08:28.992895"
[1] 36510  1716
[1] "aCM 2025-01-28 13:08:31.489254"
[1] 36510 23846


In [15]:
#looping through cell types by making ^ into a function
get_gex_SUMS <- function(cell.type, filename){
    print(paste(cell.type,Sys.time()))

    #pull out rows of gex.counts where BC Ident matches cell.type
    bcs <- names(Idents(adata_matrices)[Idents(adata_matrices) == cell.type])
    counts <- gex.counts[,colnames(gex.counts) %in% bcs]
    print(dim(counts))

    #grab and sum counts
    counts.df <- as.data.frame(rep(0,length(row.names(gex.counts))))
    row.names(counts.df) <- row.names(gex.counts)
    colnames(counts.df) <- c('counts')
    counts.df$counts = rowSums(counts)
    write.table(counts.df, filename, sep='\t',quote=FALSE)
}

In [16]:
##### NAME YOUR FILES #####
for (cell.type in unique_cell_types){
    filename <- paste(outdir, cell.type, '_gex_SoupX.RNA.counts', sep = "")
    get_gex_SUMS(cell.type, filename)
}

[1] "Fibroblast 2025-01-28 13:08:37.935452"
[1] 36510 67157
[1] "Endothelial 2025-01-28 13:08:40.570469"
[1] 36510 53650
[1] "vCM 2025-01-28 13:08:42.480201"
[1] 36510 68449
[1] "Myeloid 2025-01-28 13:08:45.856776"
[1] 36510 51355
[1] "Pericyte 2025-01-28 13:08:48.16758"
[1] 36510 22489
[1] "Endocardial 2025-01-28 13:08:49.604749"
[1] 36510  8927
[1] "Lymphoid 2025-01-28 13:08:50.954432"
[1] 36510 20021
[1] "SM 2025-01-28 13:08:52.38736"
[1] 36510  6561
[1] "Neuronal 2025-01-28 13:08:53.596673"
[1] 36510  3962
[1] "Adipocyte 2025-01-28 13:08:54.830385"
[1] 36510  1122
[1] "Epicardial 2025-01-28 13:08:56.038428"
[1] 36510  1716
[1] "aCM 2025-01-28 13:08:57.220367"
[1] 36510 23846


# TPM

In [13]:
# Gene Infos
fin.gene.info = read.table("/nfs/lab/publicdata/gencode_v38/gene_info_withExonicGeneSizes.tsv",
                           header=T)

### SOUPX correction MATRICES
dir = counts.dir
outdir = TPM.dir

# TPM function
make_tpm = function(counts, gene_sizes){
    rpk <- counts / gene_sizes
    tpm <- rpk
    for (i in 1:ncol(rpk)){
        tpm[,i] <- rpk[,i]/(sum(rpk[,i])/1e6)
    }
    return(tpm)
}

In [14]:
# Per sample TPM
# get list of files
files = list.files(dir, pattern= "_perdonor.gex_SoupX.RNA.counts")
files
# cut off file suffices to get celltype names
cells = gsub("_perdonor.gex_SoupX.RNA.counts","", files)
cells 

for (FILE in files){
    message("reading ", FILE)
    cell = cells[which(files == FILE)]
    counts = read.table(paste0(dir, FILE), row.names=1)
    #sumreads[,cell]= rowSums(counts)
    counts = subset(counts ,rownames(counts) %in% fin.gene.info$gene_name)
    message(" - Subset to: ", nrow(counts))
    gene_sizes = fin.gene.info$exonic.gene.sizes[match(rownames(counts), fin.gene.info$gene_name)]
    tpm_mat = make_tpm(counts, gene_sizes)
    tpm_mat[is.na(tpm_mat)] <- 0
    write.table(tpm_mat, paste0(outdir,  cell, "_perdonor.gex_SoupX.RNA.tpm"), sep="\t", quote=F)
    
    genes = data.frame(gene = rownames(tpm_mat), 
                   TPM = rowSums(tpm_mat))
    write.table(genes, paste0(outdir,  cell, ".tpm"), sep="\t", quote=F, 
                col.names = TRUE, row.names = FALSE)
    
    expressed.genes = genes %>%
                    dplyr::filter(TPM > 1)
    expressed.genes = as.data.frame(expressed.genes$gene)
    message(" - expressed genes: ", nrow(expressed.genes))
    write.table(expressed.genes, paste0(outdir,  cell, "_expressed.genes.TPM1.ls"), sep="\t", quote=F, 
               col.names = FALSE, row.names = FALSE)
}

[1] "aCM_perdonor.gex_SoupX.RNA.counts"        
 [2] "Adipocyte_perdonor.gex_SoupX.RNA.counts"  
 [3] "Endocardial_perdonor.gex_SoupX.RNA.counts"
 [4] "Endothelial_perdonor.gex_SoupX.RNA.counts"
 [5] "Epicardial_perdonor.gex_SoupX.RNA.counts" 
 [6] "Fibroblast_perdonor.gex_SoupX.RNA.counts" 
 [7] "Lymphoid_perdonor.gex_SoupX.RNA.counts"   
 [8] "Myeloid_perdonor.gex_SoupX.RNA.counts"    
 [9] "Neuronal_perdonor.gex_SoupX.RNA.counts"   
[10] "Pericyte_perdonor.gex_SoupX.RNA.counts"   
[11] "SM_perdonor.gex_SoupX.RNA.counts"         
[12] "vCM_perdonor.gex_SoupX.RNA.counts"

[1] "aCM"         "Adipocyte"   "Endocardial" "Endothelial" "Epicardial" 
 [6] "Fibroblast"  "Lymphoid"    "Myeloid"     "Neuronal"    "Pericyte"   
[11] "SM"          "vCM"

reading aCM_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 21237

reading Adipocyte_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 18935

reading Endocardial_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 20522

reading Endothelial_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 21498

reading Epicardial_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 19936

reading Fibroblast_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 21615

reading Lymphoid_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 21002

reading Myeloid_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 21294

reading Neuronal_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 19144

reading Pericyte_perdonor.gex_SoupX.RNA.counts

 - Subset to: 24228

 - expressed genes: 20576

reading SM_perdonor.gex_SoupX.RNA.c

In [15]:
# SAVE TPM per celltype
TPM.by.cell <- data.frame()

for (c in unique_cell_types) {
    message("Processing ", c)
    ct.TPM <- rowMeans(read.table(paste0(outdir,c,'_perdonor.gex_SoupX.RNA.tpm')), na.rm=T)
    if(length(rownames(TPM.by.cell)) == 0) {
        TPM.by.cell <- data.frame(ct.TPM)
        colnames(TPM.by.cell) <- c
    } else {
        TPM.by.cell[[c]] <- ct.TPM[rownames(TPM.by.cell)]
    }
}

dim(TPM.by.cell)
head(TPM.by.cell)

write.table(TPM.by.cell, 
            paste0(outdir, 'AllCellTypes.gex_SoupX.RNA.tpm'),
           quote=F, col.names=T, row.names=T, sep='\t')

ERROR: Error in eval(expr, envir, enclos): object 'unique_cell_types' not found
